In [1]:
import tensorflow as tf

In [2]:
len(tf.config.experimental.list_physical_devices('GPU'))

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True

tf.compat.v1.disable_eager_execution()

In [3]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

True

In [4]:
dataset = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = dataset.load_data()

In [5]:
y_train = tf.one_hot(y_train, 10, dtype=tf.uint8)
y_test = tf.one_hot(y_test, 10, dtype=tf.uint8)

In [6]:
y_train

<tf.Tensor 'one_hot:0' shape=(60000, 10) dtype=uint8>

In [7]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784 # 28*28*1

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2

In [8]:
weights = {
    'wc1': tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1], dtype=tf.float16), dtype=tf.half, trainable=True),
    'wc2': tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2], dtype=tf.float16), dtype=tf.half, trainable=True),
    'wh1': tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden], dtype=tf.float16), dtype=tf.half, trainable=True),
    'wo': tf.Variable(tf.random.normal([n_hidden, n_out], dtype=tf.float16), dtype=tf.half, trainable=True)
}

biases = {
    'bc1': tf.Variable(tf.random.normal([n_conv1], dtype=tf.float16), dtype=tf.half, trainable=True),
    'bc2': tf.Variable(tf.random.normal([n_conv2], dtype=tf.float16), dtype=tf.half, trainable=True),
    'bh1': tf.Variable(tf.random.normal([n_hidden], dtype=tf.float16), dtype=tf.half, trainable=True),
    'bo': tf.Variable(tf.random.normal([n_out], dtype=tf.float16), dtype=tf.half, trainable=True)    
}

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
def conv(x, weights, biases, strides=1):
    out = tf.nn.conv2d(tf.cast(x, tf.half), weights, padding="SAME", strides=[1, strides, strides, 1])
    out = tf.nn.bias_add(out, biases)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k=2):
    return tf.nn.max_pool(x, padding="SAME", ksize=[1, k, k, 1], strides=[1, k, k, 1])

In [10]:
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape=[-1, input_height, input_width, input_channels])
    conv1 = conv(tf.cast(x, tf.half), weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape=[-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)
    
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [11]:
drop = 0.2

train_pred = cnn(tf.cast(x_train[:5000], tf.half), weights, biases, drop)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=train_pred, labels=y_train[:5000]))

In [12]:
sess = tf.compat.v1.Session()

sess.run(tf.compat.v1.global_variables_initializer())

sess.run(train_pred)

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float16)

In [13]:
tf.compat.v1.trainable_variables()

[<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float16>,
 <tf.Variable 'Variable_1:0' shape=(5, 5, 32, 64) dtype=float16>,
 <tf.Variable 'Variable_2:0' shape=(3136, 1024) dtype=float16>,
 <tf.Variable 'Variable_3:0' shape=(1024, 10) dtype=float16>,
 <tf.Variable 'Variable_4:0' shape=(32,) dtype=float16>,
 <tf.Variable 'Variable_5:0' shape=(64,) dtype=float16>,
 <tf.Variable 'Variable_6:0' shape=(1024,) dtype=float16>,
 <tf.Variable 'Variable_7:0' shape=(10,) dtype=float16>]

In [14]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [15]:
c, _ = sess.run([cost, optimize])

FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Error while reading resource variable beta1_power from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/beta1_power/class tensorflow::Var does not exist.
	 [[node Adam/update_Variable/Cast/ReadVariableOp (defined at d:\documents\codingninjas\venv\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
	 [[Mean/_3]]
  (1) Failed precondition: Error while reading resource variable beta1_power from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/beta1_power/class tensorflow::Var does not exist.
	 [[node Adam/update_Variable/Cast/ReadVariableOp (defined at d:\documents\codingninjas\venv\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Adam/update_Variable/Cast/ReadVariableOp':
  File "c:\users\zenkor

In [ ]:
print(c)